#

------------------------------------------------------------------------

> **Important**
>
> In this section we will take a look at quantitative fundamental analysis tools provided by the “tidyquant” R language package \[@danchoTidyquantTidyQuantitative2025\] and use them to get key financial ratios of fundamental analysis for the Clorox company. This section is based on the following wonderful tutorial: @PerformanceAnalysisTidyquant

First let’s retrieve 5-year period returns from the prices adjusted for stock splits, both for Clorox and S&P500 as the baseline. Next, we combine both datasets (Clorox and S&P500 returns) using “left-join” on the “date” field. Using this dataset, we can retrieve all kind of fundamental metrics, such as **Alpha** (0.0008), **Beta** (0.414).

``` r

library(tidyverse)
```

    ── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
    ✔ dplyr     1.1.4     ✔ readr     2.1.5
    ✔ forcats   1.0.0     ✔ stringr   1.5.1
    ✔ ggplot2   3.5.1     ✔ tibble    3.2.1
    ✔ lubridate 1.9.4     ✔ tidyr     1.3.1
    ✔ purrr     1.0.4     
    ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
    ✖ purrr::%||%()   masks base::%||%()
    ✖ dplyr::filter() masks stats::filter()
    ✖ dplyr::lag()    masks stats::lag()
    ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

``` r

library(tidyquant)
```

    Registered S3 method overwritten by 'quantmod':
      method            from
      as.zoo.data.frame zoo 
    ── Attaching core tidyquant packages ─────────────────────── tidyquant 1.0.10 ──
    ✔ PerformanceAnalytics 2.0.8      ✔ TTR                  0.24.4
    ✔ quantmod             0.4.26     ✔ xts                  0.14.1── Conflicts ────────────────────────────────────────── tidyquant_conflicts() ──
    ✖ purrr::%||%()                  masks base::%||%()
    ✖ zoo::as.Date()                 masks base::as.Date()
    ✖ zoo::as.Date.numeric()         masks base::as.Date.numeric()
    ✖ dplyr::filter()                masks stats::filter()
    ✖ xts::first()                   masks dplyr::first()
    ✖ dplyr::lag()                   masks stats::lag()
    ✖ xts::last()                    masks dplyr::last()
    ✖ PerformanceAnalytics::legend() masks graphics::legend()
    ✖ quantmod::summary()            masks base::summary()
    ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

``` r

Ra <- "CLX" |>
    tq_get(get  = "stock.prices",
           from = "2020-01-01",
           to   = "2025-02-01") |>
    group_by(symbol) |>
    tq_transmute(select     = adjusted, 
                 mutate_fun = periodReturn, 
                 period     = "monthly", 
                 col_rename = "Ra")


Rb <- "^GSPC" |>
    tq_get(get  = "stock.prices",
           from = "2020-01-01",
           to   = "2025-02-01") |>
    tq_transmute(select     = adjusted, 
                 mutate_fun = periodReturn, 
                 period     = "monthly", 
                 col_rename = "Rb")

RaRb <- left_join(Ra, Rb, by = c("date" = "date"))


RaRb_capm <- RaRb %>%
    tq_performance(Ra = Ra, 
                   Rb = Rb, 
                   performance_fun = table.CAPM)
```

    Registered S3 method overwritten by 'robustbase':
      method          from     
      hatvalues.lmrob RobStatTM

``` r

RaRb_capm %>% select(symbol, Alpha, Beta)
```

    # A tibble: 1 × 3
    # Groups:   symbol [1]
      symbol  Alpha  Beta
      <chr>   <dbl> <dbl>
    1 CLX    0.0008 0.414

We can also get the **Annualized Sharpe Ratio and Returns**, **Types of Mean Return (Geometric, Arithmetic, etc.)**, **Kurtosis**, as well as **Maximum and Median Return** by following the similar steps (no need for the baseline this time).

In [ ]:
library(tidyverse)
library(tidyquant)

Ra <- "CLX" |>
    tq_get(get  = "stock.prices",
           from = "2020-01-01",
           to   = "2025-02-01") |>
    group_by(symbol) |>
    tq_transmute(select     = adjusted, 
                 mutate_fun = periodReturn, 
                 period     = "monthly", 
                 col_rename = "Ra")

Ra |>
  tq_performance(Ra = Ra, Rb = NULL, performance_fun = table.AnnualizedReturns)


# A tibble: 1 × 4
# Groups:   symbol [1]
  symbol AnnualizedReturn `AnnualizedSharpe(Rf=0%)` AnnualizedStdDev
  <chr>             <dbl>                     <dbl>            <dbl>
1 CLX              0.0381                     0.153            0.249

# A tibble: 1 × 17
# Groups:   symbol [1]
  symbol ArithmeticMean GeometricMean Kurtosis `LCLMean(0.95)` Maximum Median
  <chr>           <dbl>         <dbl>    <dbl>           <dbl>   <dbl>  <dbl>
1 CLX            0.0056        0.0031     1.20         -0.0128   0.218 0.0054
# ℹ 10 more variables: Minimum <dbl>, NAs <dbl>, Observations <dbl>,
#   Quartile1 <dbl>, Quartile3 <dbl>, SEMean <dbl>, Skewness <dbl>,
#   Stdev <dbl>, `UCLMean(0.95)` <dbl>, Variance <dbl>

#### **I. Graham Number Calculation**

The Graham Number estimates a stock’s intrinsic value using earnings per share (EPS) and book value per share (BVPS):  
**Formula**:

$$\text{Graham Number} = \sqrt{22.5 \times \text{EPS} \times \text{BVPS}}$$

-   **EPS (TTM)**: \$3.67  
-   **BVPS**: \$0.98 (Total Equity: \$121M / Shares Outstanding: 123.19M)  
-   **Calculation**:  
    $$\sqrt{22.5 \times 3.67 \times 0.98} = \sqrt{80.92} \approx \boxed{8.99}$$
-   **Interpretation**: CLX’s current stock price (~\$149.61) far exceeds the Graham Number, indicating potential overvaluation based on traditional metrics .

------------------------------------------------------------------------

#### **II. Key Financial Ratios**

| **Category** | **Ratio** | **CLX Value** | **Formula** | **Interpretation** |
|-------|---------|----------|-----------------------|------------------------|
| **Profitability** | Gross Profit Margin | 44.51% | (Revenue - COGS) / Revenue | Stable gross margins despite cost inflation, supported by cost-saving initiatives . |
|  | Operating Profit Margin | 14.32% | Operating Income / Revenue | Margin expansion driven by divestitures and operational recovery post-cyberattack . |
|  | Net Profit Margin | 6.38% | Net Income / Revenue | Lower than peers (e.g., P&G: ~18%), reflecting cyberattack recovery costs . |
|  | Return on Assets (ROA) | 11.17% | Net Income / Avg. Total Assets | Efficient asset utilization, though impacted by divestitures . |
|  | Return on Equity (ROE) | 276.11% | Net Income / Avg. Shareholders’ Equity | Artificially high due to low equity base (\$121M) . |
| **Liquidity** | Current Ratio | 0.94 | Current Assets / Current Liabilities | Below industry norms (1.5–3.0), indicating liquidity strain . |
|  | Quick Ratio | 0.52 | (Cash + Receivables) / Current Liabilities | Limited near-term liquidity, with heavy reliance on receivables . |
| **Solvency** | Debt-to-Equity Ratio | 25.55 | Total Liabilities / Shareholders’ Equity | Extremely high leverage (vs. industry median ~1.0), raising solvency concerns . |
|  | Debt-to-Assets Ratio | 98% | Total Liabilities / Total Assets | Nearly all assets financed by debt, increasing bankruptcy risk . |
|  | Interest Coverage Ratio | ~11.7 | Operating Income / Interest Expense | Strong coverage due to operational recovery, but debt levels remain risky. |
| **Efficiency** | Asset Turnover Ratio | 1.25 | Revenue / Avg. Total Assets | Efficient asset use, though lower than pre-cyberattack levels . |
|  | Inventory Turnover Ratio | 6.38 | COGS / Avg. Inventory | Healthy turnover, but down from 7.57 in 2020 . |
|  | Receivables Turnover Ratio | ~11.05 | Revenue / Avg. Accounts Receivable | Faster collections than industry median (~30 days) . |
| **Valuation** | P/E Ratio | 40.50 | Price per Share / EPS | High vs. peers (PG: 26.66, CL: 24.39), reflecting recovery optimism . |
|  | P/B Ratio | 61.19 | Price per Share / Book Value per Share | Extremely high, driven by low equity base and market premium . |
|  | P/S Ratio | 2.59 | Price per Share / Revenue per Share | Aligns with sector norms (Consumer Staples avg: ~2.5–3.0) . |
|  | Dividend Yield | 3.26% | Annual Dividend / Price per Share | Attractive yield, supported by 48-year dividend growth streak . |

------------------------------------------------------------------------

#### **III. Key Insights**

1.  **Profitability**: Margins are recovering post-cyberattack, but net margins lag competitors. Gross margin improved to 43.8% in Q2 FY25, with cost savings offsetting inflation .  
2.  **Liquidity Risks**: Low current (0.94) and quick (0.52) ratios highlight liquidity challenges, though improved cash flow (\$401M YTD) provides relief .  
3.  **Debt Concerns**: Debt-to-equity (25.55) and debt-to-assets (98%) ratios signal high leverage, though interest coverage (11.7x) remains manageable .  
4.  **Valuation**: Elevated P/E (40.5) and P/B (61.19) suggest market optimism about Clorox’s recovery, but Graham Number (\$8.99) implies overvaluation .  
5.  **Dividend Strength**: A 3.26% yield with consistent growth underscores Clorox’s defensive appeal, despite a high payout ratio (133%) .

------------------------------------------------------------------------

#### **IV. Conclusion**

Clorox’s fundamentals reflect a company in recovery, balancing strong brand equity and dividend reliability with significant debt and valuation risks. While operational improvements (e.g., margin expansion, ERP transition) are promising, the stock’s premium pricing and leverage warrant caution. Investors should monitor debt reduction and margin sustainability in 2025 .